In [1]:
import math
import random as rnd
import numpy as np
from scipy import stats
import time

#Moves list - will be ammended for each state as some are not possible in some states!
MOVES=[[0,1],[0,2],[0,3],[0,4],[1,0],[1,1],[1,2],[1,3],[1,4],[1,5],[1,6],[1,-1],[1,-2],
       [1,-3],[1,-4],[2,2], [2,3], [2,4],[2,5],[2,6],'Dudo', 'Calza']

#import Rule based Agent
%run RuleBasedAgent.ipynb

In [2]:
"""
This function checks what the possible moves 
from the current state are.
"""

def check_possible_moves(node, moves, no_dice, roll):
    possible_moves = []
    threshold_for_aces = 0.25
    threshold_for_vals = 0.45
    
    #If first bid!
    if node.previous== [0,0]:
        for i in moves:
            if isinstance (i, list): #start with anything apart from 1's and quantity 0
                if i[0] != 0 and i[1]>1:
                    possible_moves.append(i) 

    #Not first bid
    else:
        if isinstance(node.previous, list): #If its a value:
            
            #From an aces bid
            if node.previous[1] == 1: #Aces:
                if node.previous[0]<threshold_for_aces*no_dice: #the bid quant is not over 70% of the dice
                    for val in range(2,7): 
                        move = [(node.previous[0]*2)+1, val, 0]
                        possible_moves.append(move)
                    for x in moves:
                        if isinstance(x, list):
                            if x[0] != 0 and x[1] == 0: #+1 or +2 of the same value
                                possible_moves.append(x)
                        else:
                            possible_moves.append(x) #Dudo calza aces
                else: #not within reasonable probabilities
                    for x in moves:
                        if isinstance(x, str):
                            possible_moves.append(x) #only append 'Dudo' or 'Calza'
                            
            else: #Any other value
                for x in moves:
                    if isinstance(x, list):
                        if node.previous[0]<threshold_for_vals*no_dice: #the bid quant is not over 70% of the dice
                            if x[0] <=1:
                                if node.previous[1] + x[1] <= 6:
                                    if node.previous[1]+ x[1] > 1: #can't say 1,2 and then 2-1 ?¿
                                        possible_moves.append(x) 
                    else:
                        possible_moves.append(x) #append 'Dudo' or 'Calza'
                        
                change_to_aces = 'Aces' #changing from vals to aces
                change_to_aces2 = 'Aces2' #changing from vals to aces +1
                possible_moves.append(change_to_aces)
                possible_moves.append(change_to_aces2)
        else:
            possible_moves = [] #means its a terminal node!
            
    return possible_moves
    

"""
This function evaluates the score of a 
rollout - after dudo or calza.
"""

def score(node, scoring,previous_bid, rollout_player_parent, rollout_player, mcts_player, players, no_players, dice_player, 
          dice_state_opponents, all_dice, Dudo = False, Calza = False):
    
    if previous_bid[1] == 1:
        dice_name = 'Aces'
    elif previous_bid[1] == 2:
        dice_name = 'Twos'
    elif previous_bid[1] == 3:
        dice_name = 'Threes'
    elif previous_bid[1] == 4:
        dice_name = 'Fours'
    elif previous_bid[1] == 5:
        dice_name = 'Fives'
    elif previous_bid[1] == 6:
        dice_name = 'Sixes'
        
    dice_revealed = count(no_players, dice_player, dice_state_opponents,mcts_player, 
                          players, all_dice)

    lost_die = gained_die = False
    
    if Dudo == True:
        #Aces
        if dice_name == 'Aces':
            
            #Wrong Dudo 
            if dice_revealed['Aces']>= previous_bid[0]: 
                if rollout_player == mcts_player: #from mcts player
                    lost_die = True
                    
            #Correct Dudo        
            else: 
                if rollout_player_parent == mcts_player: #Correct Dudo from someone to mcts player
                    lost_die = True
        #Other values
        else:
            
            #Wrong Dudo
            if dice_revealed['Aces']+dice_revealed[dice_name] >= previous_bid[0]: 
                if rollout_player == mcts_player:
                    lost_die = True
                    
            #Correct Dudo        
            else: 
                if rollout_player_parent == mcts_player: #Correct Dudo from someone to mcts player
                    lost_die = True
                    
    elif Calza == True:
        if dice_name == 'Aces':
            if dice_revealed['Aces'] == previous_bid[0]:
                if rollout_player == mcts_player:
                    gained_die = True
            else:
                if rollout_player == mcts_player:
                    lost_die = True
        else:
            if dice_revealed['Aces']+ dice_revealed[dice_name] == previous_bid[0]:
                if rollout_player == mcts_player:
                    gained_die = True
            else:
                if rollout_player == mcts_player:
                    lost_die = True
                    
    if lost_die == True:
        score = scoring[0]
    elif gained_die == True:
        score = scoring[2]
    else:
        score = scoring[1]

    return score

"""
This function chooses the best child
of a node considering their ucbs.
"""

def best_child(node,scalar):
    bestchildren=[]
    best_ucb = float('-inf')
    for c in node.children:
        ucb = c.ucb1(scalar)
        if ucb>best_ucb:
            bestchildren = [c]
            best_ucb= ucb
        if ucb==best_ucb:
            bestchildren.append(c)
    best_node = rnd.choice(bestchildren)
    return best_node

"""
Assigns random rolls to all players
"""
def set_rolls(no_players, dice_player, dice_state_opponents, mcts_player, players):
    all_rolls = {}
    all_rolls[mcts_player] = dice_player
    
    for i in dice_state_opponents:
        opponents_roll = [rnd.randint(1,6) for i in range(dice_state_opponents[i])]
        all_rolls[i] = opponents_roll
    return all_rolls

"""
Counts how many vals are there for each dice
possibility - this eases assigning winner and loser
"""

def count(no_players, dice_player, dice_state_opponents, mcts_player, players, all_dice):
    Aces = Twos = Threes = Fours = Fives = Sixes = 0
    dice_count = [Aces, Twos, Threes, Fours, Fives, Sixes]
    dice_val_names = ['Aces', 'Twos', 'Threes', 'Fours', 'Fives', 'Sixes']
    dice_vals = (1,2,3,4,5,6)
    count = {}
    #count the dice
    for p in all_dice:
        for x in range(0,6):
            dice_count[x] += all_dice[p].count(x+1)
    for y in range(len(dice_vals)): 
        count[dice_val_names[y]] = dice_count[y]
    return count

"""
Calculated the number of rollouts optimal to the number of opponents dice
"""
def calc_rollouts(total_dice, player_dice, dice_state_opponents,random=True):
    if random == True:
        opponent_dice=  total_dice - len(player_dice)
        CR = math.factorial(6 + opponent_dice -1) / (math.factorial(opponent_dice) * math.factorial (6-1))
        if CR > 10000:
            return 10000
        else:
            if CR*1.2 < 10000:
                return CR*1.2
            else:
                return CR
    else:
        CR = 1
        for x in dice_state_opponents:
            CR = CR * (math.factorial(6 + dice_state_opponents[x] -1) / (math.factorial(dice_state_opponents[x]) * math.factorial (6-1)))#####
        if CR > 3500:
            return 3500
        else:
            return CR
"""
Keeps searching down the tree randomly until finding a
terminal node and assigns it a rollout score.
"""
def rollout(current_node, scalar, scoring,mcts_player, players, no_players, dice_player, 
            dice_state_opponents, total_dice, moves, Random):
    history = current_node.history.copy()
    rolls = set_rolls(len(players), dice_player, dice_state_opponents, mcts_player, players)
    rollout_current = current_node
    while rollout_current.terminal==False:
        if len(rollout_current.children)==0:
            if Random == True:
                rollout_current.expand_node()
                rollout_current = rnd.choice(rollout_current.children)
            else:
                if rollout_current.player == mcts_player:
                    rollout_current.expand_node()
                    rollout_current = rnd.choice(rollout_current.children)
                    history.append(rollout_current.previous)
                else:
                    opponent_roll = rolls[rollout_current.player]                    
                    agent = rule_based_agent(opponent_roll,rollout_current.previous, 
                                             rollout_current.history,total_dice, players)
                    bid = agent.make_bid()
                    if bid == 'Dudo' or bid == 'Calza':
                        bid_before_call = rollout_current.previous
                    else:
                        bid_before_call = None
                    rollout_current = Node(moves, rollout_current.next_player,mcts_player, players,
                                          dice_player, total_dice, history,parent = rollout_current, 
                                           previous_bid=bid, bid_before_call = bid_before_call)
                    history.append(rollout_current.previous)
                    
    Dudo = Calza = False
    if rollout_current.previous == 'Dudo':
        Dudo = True
    elif rollout_current.previous == 'Calza':
        Calza = True
        
    value = score(current_node,scoring,rollout_current.bid_before_call, rollout_current.parent.previous_player, 
                      rollout_current.parent.player,mcts_player, players,no_players, 
                      dice_player, dice_state_opponents, rolls, Dudo = Dudo, Calza = Calza)

    return value


In [3]:
class Node:
    def __init__(self, moves, player, mcts_player, players, dice_player, no_dice, history, 
                 parent = None, previous_bid = None, bid_before_call=None):
             
        # parent and child nodes
        self.parent = parent
        self.children = []
        self.value = 0
        self.visits = 0
        
        #Player names to every node to be able to assign a loss score to the right player
        self.Players = players
        self.player = player
        self.mcts_player = mcts_player
        
        #previous player to arrange scores for dudo
        indx = self.Players.index(self.player)
        if indx == 0:
            self.previous_player = self.Players[-1]
        else:
            self.previous_player = self.Players[indx-1]
        
        #next player to create children with the correct player name
        player_indx = self.Players.index(player)
        if player_indx == (len(self.Players)-1):
            self.next_player = self.Players[0]
        else:
            self.next_player = self.Players[player_indx+1]
    
        #Previous bid       
        if previous_bid == None:
            self.previous = [0,0]
        else:
            self.previous = previous_bid
            
        #Terminal State            
        if self.previous == 'Dudo' or self.previous == 'Calza':
            self.terminal = True
        else:
            self.terminal = False
        
        #Possible moves and bid before call(dudo/exactly)
        self.no_dice = no_dice
        self.roll = dice_player
        self.moves = check_possible_moves(self, moves, self.no_dice, self.roll)
        
        #When previous bid is dudo, we store the previous previous bid here to score rollout!
        self.bid_before_call = bid_before_call
        self.history = history
        self.history.append(self.previous)
        
        
    # generate the children of this node.
    def expand_node(self):
        for i in range(len(self.moves)):
            
            #Numerical moves!
            if isinstance(self.moves[i], list):
                if len(self.moves[i]) < 3:
                    next_bid = [self.previous[0]+self.moves[i][0], self.previous[1]+self.moves[i][1]]
                else:
                    next_bid = [self.moves[i][0], self.moves[i][1]]

                new_child=Node(MOVES, self.next_player, self.mcts_player, self.Players,
                               self.roll,self.no_dice,self.history, parent = self, previous_bid = next_bid)
                self.children.append(new_child)
                
            #String moves! (Dudo Calza or Aces)
            else:
                
                #Aces
                if self.moves[i] == 'Aces' or self.moves[i] == 'Aces2':
                    if self.moves[i] == 'Aces':
                        t = 0
                    else:
                        t = 1
                    aces_bid = [(math.ceil(self.previous[0]/2)+t),int(1)] 
                    new_child= Node(MOVES, self.next_player, self.mcts_player, self.Players,
                                    self.roll,self.no_dice, self.history, parent = self,previous_bid = aces_bid)
                    self.children.append(new_child)
                    
                #Dudo or Calza    
                else:
                    new_child = Node(MOVES, self.next_player, self.mcts_player, self.Players,
                                     self.roll,self.no_dice, self.history,parent = self, previous_bid = self.moves[i], 
                                     bid_before_call = self.previous)
                    self.children.append(new_child)
        
    # update this node via backpropagation.
    def update(self, value):
        while self!=None:
            self.visits+=1
            self.value += value
            self=self.parent

    # return the UCB1 value of this node.
    def ucb1(self, scalar):
        Min = Max = False
        if self.parent.player == self.mcts_player:
            Max = True
        else:
            Min = True
            
        #Return infinity if never been visited!
        if self.parent.visits ==0 or  self.visits == 0:
            return float('inf')
        
        else:
            N = self.parent.visits
            ni = self.visits
            vi = self.value/self.visits
            k = (np.log(N)/ni)      
        if Max == True:
            UCB = vi + (scalar*(math.sqrt(k)))
        elif Min == True:
            UCB = - vi + (scalar*((math.sqrt(k))))

        return UCB

In [4]:
"""
This class contains the template for MCTSPlayer, which
chooses a move based on a Monte-Carlo Tree search using
UCB1.
"""
class MCTSPlayer():
        
    def __init__(self, moves, scalar, no_players, dice_player, no_dice, Players,
                 player, dice_state_opponents, history, scoring, previous_bid = None, Random=True,
                 rollouts=2000, smart_rollouts = False):
        
        if smart_rollouts == False:
            self.rollouts = rollouts
        else: 
            self.rollouts = calc_rollouts(no_dice, dice_player, dice_state_opponents, Random)
        self.moves_for_node = moves
        self.previous = previous_bid
        self.no_players = no_players
        self.dice_player = dice_player
        self.dice_state_opponents = dice_state_opponents
        self.no_dice = no_dice
        self.Players = Players #list of player names
        self.player = player #current_player name 
        self.mcts_player= player
        index = self.Players.index(self.player)
        self.previous_round_player = self.Players[index-1] 
        self.history = history.copy()
        self.random = Random
        self.scalar = scalar
        self.scoring = scoring
      
    def chooseMove(self):
        self.root_node = Node(self.moves_for_node, self.player, self.mcts_player,self.Players,
                              self.dice_player, self.no_dice, self.history, previous_bid = self.previous) #self.player will be the previous_player!
        
        if self.history == []:
            self.rollouts = 100 #First bid isn't so important - this saves unncessary computational time
        
        for i in range(int(self.rollouts)):
            if self.root_node.children == []: #leaf node! 
                self.root_node.expand_node()
                
            #find a leaf node!
            current_node = self.root_node
            while current_node.children != []: 
                current_node = best_child(current_node, scalar)
                
            if current_node.visits == 0:
                #Rollout A 
                rollout_value = rollout(current_node, self.scalar, self.scoring,self.mcts_player, self.Players, 
                                    self.no_players, self.dice_player, self.dice_state_opponents,
                                       self.no_dice, self.moves_for_node, self.random)
        
                #Backpropagation!
                current_node.update(rollout_value)
            
            #if visited before - expand and then rollout!
            else:
                current_node.expand_node()
                
                if current_node.children != []: #if after expanding it has no children its a terminal state
                    #Else, rollout a child
                    current_node = rnd.choice(current_node.children)
                
                #Rollout!
                rollout_value = rollout(current_node, self.scalar, self.scoring, self.mcts_player, self.Players, 
                                    self.no_players, self.dice_player, self.dice_state_opponents,
                                       self.no_dice, self.moves_for_node,self.random)
                
                #Backpropagation!
                current_node.update(rollout_value)


        # select move        
        best_kid = []
        best_value = float('-inf')
        for i in self.root_node.children:
            if i.visits > 0:
                if i.value/i.visits > best_value:
                    best_kid = [i]
                    best_value = i.value/i.visits
                elif i.value == best_value:
                    best_kid.append(i)
            else:
                continue

        #return the chosen move
        move = rnd.choice(best_kid)
        move_action = move.previous
        
        return move_action
    